# Trabalho Prático 01 - Computação Gráfica SCC0650

##### Luiz Fernando Rabelo (11796893) e Matheus Bermudes Viana (11849797)

### Importação de Bibliotecas

No primeiro passo, importamos as bibliotecas padrão vistas em aula (OpenGL, _glfw_ para gerenciamento das janelas e _numpy_ para operações matemáticas e para manipulação de vetores e matrizes).

In [44]:
import glfw
from OpenGL.GL import *
import OpenGL.GL.shaders
import numpy as np

### Inicialização da Janela GLFW

Inicializamos a janela com uma largura de 1000 e altura de 800, com o título "Trabalho Prático 1", determinando que inicialmente ela não será exibida. Optamos por passar parâmetros nulos para _monitor_ e _share_.

In [45]:
glfw.init()
glfw.window_hint(glfw.VISIBLE, glfw.FALSE)
window = glfw.create_window(1000, 800, 'Trabalho Prático 1', None, None)
glfw.make_context_current(window)

### Definição de Código GLSL para Vertex Shader e Fragment Shader

A fim de possibilitarmos as transformações e configurações de cores, atribuímos códigos GLSL para `vertex_code` e `fragment_code`:

In [46]:
vertex_code = """
        attribute vec2 position;
        uniform mat4 mat_transformation;
        void main(){
            gl_Position = mat_transformation * vec4(position,0.0,1.0);
        }
        """

fragment_code = """
        uniform vec4 color;
        void main(){
            gl_FragColor = color;
        }
        """

### Requisição de slot para a GPU para Vertex e Fragment Shaders

In [47]:
program  = glCreateProgram()
vertex   = glCreateShader(GL_VERTEX_SHADER)
fragment = glCreateShader(GL_FRAGMENT_SHADER)

### Associação do Código Fonte aos Slots Solicitados

In [48]:
glShaderSource(vertex, vertex_code)
glShaderSource(fragment, fragment_code)

### Compilação do Vertex e Fragment Shaders

In [49]:
# Compilação do Vertex Shader:
glCompileShader(vertex)
if not glGetShaderiv(vertex, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(vertex).decode()
    print(error)
    raise RuntimeError("Erro de compilacao do Vertex Shader")

# Compilação do Fragment Shader:
glCompileShader(fragment)
if not glGetShaderiv(fragment, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(fragment).decode()
    print(error)
    raise RuntimeError("Erro de compilacao do Fragment Shader")

### Linkagem do Programa

In [50]:
# Associação dos Shaders complilados ao programa principal:
glAttachShader(program, vertex)
glAttachShader(program, fragment)

# Construção do programa:
glLinkProgram(program)
if not glGetProgramiv(program, GL_LINK_STATUS):
    print(glGetProgramInfoLog(program))
    raise RuntimeError('Linking error')
    
# Declaração do programa atual como o default:
glUseProgram(program)

### Preparação dos Dados que Serão Enviados para a GPU

In [51]:

#!!!

# Obtenção dos vértices de cada objeto:
# TODO:
# v1 = create_vertex_object1()
# v2 = create_vertex_object2()
# ...
# vn = create_vertex_objectn()

# Cálculo da quantidade de vértices resultante:
# TODO:
# soma = len(v1) + len(v2) + ... + len(vn)

# Preparando espaço para soma vértices usando 2 coordenadas (x, y):
# TODO:
# substituir 3 por soma e colocar 1 laço por objeto para atualizar os vértices
vertices = np.zeros(3, [('position', np.float32, 2)])
# Prenchendo as coordenadas de cada vértices:
vertices['position'] = [
    (0.0, +0.5),
    (-0.5, -0.5),
    (+0.5, -0.5)
]

#!!!


### Requisição de Slot de Buffer para a GPU e Envio dos Vértices

In [52]:
# Requisição de slot de buffer da GPU:
buffer = glGenBuffers(1)

# Determinação que o buffer atual é o default:
glBindBuffer(GL_ARRAY_BUFFER, buffer)

# Envio dos dados:
glBufferData(GL_ARRAY_BUFFER, vertices.nbytes, vertices, GL_DYNAMIC_DRAW)
glBindBuffer(GL_ARRAY_BUFFER, buffer)

### Associação das Variáveis do GLSL com os Dados:

In [53]:
# Definição do byte inicial e do offset de dados:
stride = vertices.strides[0]
offset = ctypes.c_void_p(0)

# Definição da variável position no vertex shader:
loc = glGetAttribLocation(program, "position")
glEnableVertexAttribArray(loc)

# Indicação do conteúdo para a variável position para a GPU:
glVertexAttribPointer(loc, 2, GL_FLOAT, False, stride, offset)

# Obtenção da localização da variável color, para que possa ser alterada:
loc_color = glGetUniformLocation(program, "color")

### Definição de Constantes Associadas ao Teclado

A fim de deixar o código mais legível, associamos alguns valores numéricos de códigos de teclas a constantes de nomes correspondentes. Definimos:

In [54]:
# Letras e seus códigos:
W_CODE = 87  # pessoa azul cima
A_CODE = 65  # pessoa azul esquerda
S_CODE = 83  # pessoa azul baixo
D_CODE = 68  # pessoa azul direita
I_CODE = 73  # pessoa rosa cima
J_CODE = 74  # pessoa rosa esquerda
K_CODE = 75  # pessoa rosa baixo
L_CODE = 76  # pessoa rosa direita

# Setas e seus códigos:
RIGHT_CODE = 262  # rotação dos cataventos direção 1
LEFT_CODE  = 263  # rotação dos cataventos direção 2
DOWN_CODE  = 264  # escala vertical menor
UP_CODE    = 265  # escala vertical maior

### Definição de Constantes Associadas à Posições

In [55]:
# Limites das posições em que as pessoas podem ocupar:
PERSON_MAX_X = +0.7
PERSON_MIN_X = -0.3
PERSON_MAX_Y = +0.7
PERSON_MIN_Y = -0.3

### Função para Capturar Eventos de Teclado


In [56]:
def key_event(window, key, scancode, action, mods):
    global px_blue, py_blue, px_pink, py_pink, s

    # # Atualização do px da pessoa azul:
    # if key == D_CODE and px_blue <= PERSON_MAX_X - 0.01: px_blue += 0.01  # direita
    # if key == A_CODE and px_blue >= PERSON_MIN_X + 0.01: px_blue -= 0.01  # esquerda

    # # Atualização do py da pessoa azul:
    # if key == W_CODE and py_blue <= PERSON_MAX_Y - 0.01: py_blue += 0.01  # cima
    # if key == S_CODE and py_blue >= PERSON_MIN_Y + 0.01: py_blue -= 0.01  # baixo

    # # Atualização do px da pessoa rosa:
    # if key == L_CODE and px_pink <= PERSON_MAX_X - 0.01: px_pink += 0.01  # direita
    # if key == J_CODE and px_pink >= PERSON_MIN_X + 0.01: px_pink -= 0.01  # esquerda

    # # Atualização do py da pessoa rosa:
    # if key == I_CODE and py_pink <= PERSON_MAX_Y - 0.01: py_pink += 0.01  # cima
    # if key == K_CODE and py_pink >= PERSON_MIN_Y + 0.01: py_pink -= 0.01  # baixo

    # if(key == 265 and s < 2.0): s += 0.01 # UP arrow
    # if(key == 264 and s > 0.5): s -= 0.01 # Down arrow  

    # if(key == 262): angle2 += 0.02 # Right arrow
    # if(key == 263): angle2 -= 0.02 # Left ar


### Exibição da Janela

In [57]:
glfw.show_window(window)

### Loop Principal da Janela

Enquanto a janela não for fechada, executamos o nosso laço principal:

In [58]:
while not glfw.window_should_close(window):
    # funcao interna do glfw para gerenciar eventos de mouse, teclado, etc
    glfw.poll_events() 

    # limpa a cor de fundo da janela e preenche com outra no sistema RGBA
    glClear(GL_COLOR_BUFFER_BIT)
    
    # definindo a cor da janela      
    glClearColor(.1, .2, .3, 1.0)

    mat_translation = np.array([    1.0, 0.0, 0.0, 0.01, 
                                    0.0, 1.0, 0.0, 0.01, 
                                    0.0, 0.0, 1.0, 0.0, 
                                    0.0, 0.0, 0.0, 1.0], np.float32)

    loc = glGetUniformLocation(program, 'mat_transformation')
    glUniformMatrix4fv(loc, 1, GL_TRUE, mat_translation)

    glDrawArrays(GL_TRIANGLES, 0, len(vertices))

    # gerencia troca de dados entre janela e o OpenGL
    glfw.swap_buffers(window)


### Finalização da Execução

Quando a janela for fechada, o laço principal será interrompido e podemos finalizar o sistema de janela _GLFW_:

In [59]:
glfw.terminate()